# DRのガウス過程を用いたバージョンの実装

## パッケージの導入

In [20]:
import lightgbm as lgb
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import log_loss, mean_squared_error, roc_auc_score
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_curve,auc
from sklearn.neighbors import NearestNeighbors


%matplotlib inline
# その他設定
pd.set_option('display.max_columns', 100)
plt.style.use('seaborn-darkgrid')
font = fm.FontProperties(fname='./TakaoPGothic.ttf')

dataset_url = 'https://raw.githubusercontent.com/iwanami-datascience/vol3/master/kato%26hoshino/q_data_x.csv'  
df = pd.read_csv(dataset_url)  
print(df.shape)

ModuleNotFoundError: No module named 'seaborn'

## データセットづくり

In [26]:
y_col = 'gamesecond'
z_col = 'cm_dummy'
x_cols = [
    col for col in df.columns if col not in [
        'gamecount', 'gamedummy', 'gamesecond', 'cm_dummy', 
        'pred_lgb', 'pred_lr', 
        'z0_reg_pred_lgb', 'z1_reg_pred_lgb',
        'z0_reg_pred_rg', 'z1_reg_pred_rg'
    ]
]
df_x_all = df[x_cols]
df_y_all = df[y_col]
df_x_all.shape, df_y_all.shape
reg_preds_lgb = np.zeros(df.shape[0])
reg_preds_rg = np.zeros(df.shape[0])

((10000, 31), (10000,))

In [27]:
for z in [0, 1]:
    print('z={}'.format(z))
    df_tmp = df.loc[df[z_col] == z]
    df_x = df_tmp[x_cols]
    df_y = df_tmp[y_col]
    print(df_x.shape, df_y.shape)

z=0
(5856, 31) (5856,)
z=1
(4144, 31) (4144,)


## ガウス過程回帰モデルの導入

In [2]:
import GPy

In [7]:
kernel = GPy.kern.RBF(input_dim=1, variance=1, lengthscale=0.2)

np.random.seed(seed=123)
N_sim = 100
x_sim = np.linspace(-1, 1, N_sim)
x_sim = x_sim[:, None]
mu = np.zeros(N_sim)
cov = kernel.K(x_sim, x_sim)
y_sim = np.random.multivariate_normal(mu, cov, size=20)

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
for i in range(20):
    ax.plot(x_sim[:], y_sim[i,:])
fig.savefig('output/fig1.png')

In [4]:
import numpy as np


In [6]:
print(np.__version__)

## 回帰モデルの式作成

## DRの作成